In [2]:
import re
import pandas as pd

from fuzzywuzzy import fuzz


from collections import defaultdict 

In [3]:
f1 = 'D:\\projects\\_external_files\\cod_files\\afg_adminboundaries_tabulardata.xlsx'
f2 = 'D:\\projects\\_external_files\\cod_files\\ukr_adminboundaries_tabulardata.xlsx'
f3 = 'D:\\projects\\_external_files\\cod_files\\tur_adminboundaries_tabulardata.xlsx'
f4 = 'D:\\projects\\_external_files\\cod_files\\syr_adminboundaries_tabulardata.xlsx'

files = [f1, f2, f3] #, f4]

In [3]:
#def just_melt(df):
#    required_columns = [c for c in df.columns if 'adm' in c]
#    print(required_columns)
#    melted_df = df[required_columns].melt(id_vars=['adm2_pcode','adm1_pcode','adm0_pcode'], var_name='adm', value_name='location_name').copy()
#    return melted_df

#df = process_cods(f1)
#df = standardize_column_names(df)
#df_new_loc = just_melt(df)

In [4]:
def build_pcode_hierarchy(df):
    required_columns = [c for c in df.columns if 'adm' in c]
    print(required_columns)
    melted_df = df[required_columns].melt(id_vars=['adm2_pcode','adm1_pcode','adm0_pcode'], var_name='adm', value_name='location_name').copy()
    return melted_df

df = process_cods(f1)
df = standardize_column_names(df)
df_new_loc = just_melt(df)
df_new_loc['my_pcode'] = df_new_loc['location_name'].apply(get_pcode_from_location)

NameError: name 'process_cods' is not defined

In [301]:
df_new_loc.to_csv("c://temp//check.csv", encoding='utf-8-sig')

In [4]:
def get_all_cols_for_level(df, lvl=0):

    return_df = pd.DataFrame(columns=['pcode','adm','location_name','adm_lvl'])
    #lvl_patterns={}

    while lvl >= 0:
        column_levels = [c for c in df.columns if str(lvl) in c]
        if len(column_levels) == 0:
            lvl -= 1
            continue

        
        pcodes = []
        names = []
        others = []
    
        for col in column_levels:
            print(col)
            pcode_match = re.match(r'(adm)(\d+)_pcode', col.lower())
            name_match = re.match(r'(adm)(\d+)_([a-z]{2,3})', col.lower())
            alt_match = re.match(r'(adm)(\d+)(alt)(\d+)_([a-z]{2,3})', col.lower())
            if pcode_match:
                pcodes.append(pcode_match.group(0))
            elif name_match:
                names.append(name_match.group(0))
            elif alt_match:
                #throw away alts for now
                pass
            else:
                others.append(col)
    
        #sanity checks
        if len(pcodes) != 1:
            print(f"unexpected condition - pcodes len should be 1. {pcodes}")
            print(pcodes)
        if len(names) < 1:
            print(f"unexpected condition - names len should be at least 1. {names}")
        if len(others) > 0:
            print(f"unexpected condition, but not fatal - others len should be empty. {others}")

    
        required_columns = names
        required_columns.extend(pcodes)
        
    
        # Melting the DataFrame to combine columns into rows
        melted_df = df[required_columns].melt(id_vars=pcodes[0], var_name='adm', value_name='location_name').copy()
        melted_df['adm_lvl'] = lvl
        melted_df = melted_df.rename(columns={pcodes[0]:'pcode'})


        return_df = pd.concat([return_df, melted_df])

        lvl -= 1

    #print(lvl_patterns)
    return_df['pcode_prefix'] = return_df['pcode'].apply(lambda x: x[0:2])
    
    return_df['lang_code'] = return_df['adm'].apply(lambda x: x.split('_')[1])
    country=''
    country = return_df['location_name'][(return_df['adm_lvl'] == 0) & (return_df['lang_code'] == 'en')].tolist()[0]
    return_df['country'] = country
    return_df = return_df.drop_duplicates().reset_index()
    return_df = return_df[return_df['location_name'].isna() == False]

    #remove common variations in names that can cause misses
    return_df['location_normalized'] = return_df['location_name'].str.lower()
    return_df['location_normalized'] = return_df['location_normalized'].apply(lambda x:  re.sub(r'[^a-zA-Z]', '', x))
                                                                              
                                                

    
    return return_df[['country','pcode_prefix','location_name','location_normalized','pcode','adm_lvl','lang_code']]



In [5]:
def standardize_column_names(df, case='lower'):
    cols = df.columns
    new_columns={}
    if case == 'lower':
        for c in cols:
            new_columns[c] = c.lower()
    elif case == 'upper':
        for c in cols:
            new_columns[c] = c.upper()

    return df.rename(columns=new_columns)
    


    

In [6]:
def process_cods(f):
    preferred_level = 'ADM3'
    backup_level = 'ADM2'

    # Create an ExcelFile object
    xls = pd.ExcelFile(f)
    sheet_names = xls.sheet_names

    if preferred_level in sheet_names:
        df = pd.read_excel(xls, sheet_name=preferred_level)
    else:
        df = pd.read_excel(xls, sheet_name=backup_level)

    return df


        

In [7]:
df_location = pd.DataFrame(columns=['country','pcode_prefix','location_name','pcode','adm_lvl','lang_code'])
for f in files:
    print(f)

    #try to access the ADM3 tab and load to a df
    df = process_cods(f)
    df = standardize_column_names(df)
    df_new_loc = get_all_cols_for_level(df, lvl=4)


    df_location = pd.concat([df_location, df_new_loc])


    print(df_location.shape)

df_location


D:\projects\_external_files\cod_files\afg_adminboundaries_tabulardata.xlsx
adm2_en
adm2_da
adm2_pcode
adm2_ref
adm2alt1_en
adm2alt2_en
adm2alt1_da
adm2alt2_da
adm2alt1_en
adm2alt1_da
adm1_en
adm1_da
adm1_pcode
adm0_en
adm0_da
adm0_pcode
(872, 7)
D:\projects\_external_files\cod_files\ukr_adminboundaries_tabulardata.xlsx
adm3_en
adm3_ua
adm3_ru
adm3_pcode
adm3_ref
adm3alt1_en
adm3alt2_en
adm3alt1_ua
adm3alt2_ua
adm3alt1_ru
adm3alt2_ru
adm3alt2_en
adm3alt2_ua
adm3alt2_ru
adm2_en
adm2_ua
adm2_ru
adm2_pcode
adm3alt1_en
adm3alt1_ua
adm3alt1_ru
adm1_en
adm1_ua
adm1_ru
adm1_pcode
adm0_en
adm0_ua
adm0_ru
adm0_pcode
(6680, 7)
D:\projects\_external_files\cod_files\tur_adminboundaries_tabulardata.xlsx
adm2_tr
adm2_en
adm2_pcode
adm1_tr
adm1_en
adm1_pcode
adm0_tr
adm0_en
adm0_pcode
(8790, 7)


,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code,location_normalized
0,Afghanistan,AF,Khash,AF1705,2,en,khash
1,Afghanistan,AF,Hazrat-e-Sultan,AF2002,2,en,hazratesultan
2,Afghanistan,AF,Pul-e-Alam,AF0501,2,en,pulealam
3,Afghanistan,AF,Mata Khan,AF1202,2,en,matakhan
4,Afghanistan,AF,Gosfandi,AF2206,2,en,gosfandi
...,...,...,...,...,...,...,...
2105,TURKEY,TU,YALOVA,TUR077,1,en,yalova
2106,TURKEY,TU,YOZGAT,TUR066,1,en,yozgat
2107,TURKEY,TU,ZONGULDAK,TUR067,1,en,zonguldak
2108,TURKEY,TU,TÜRKİYE,TUR,0,tr,trkiye


In [8]:
df_location['lvl_pcode_len'] = df_location['pcode'].apply(lambda x:len(x))
df_pcode_struct = df_location[['pcode_prefix','adm_lvl','lvl_pcode_len']].copy().drop_duplicates().reset_index(drop=True)
pivot_df = df_pcode_struct.pivot(index='pcode_prefix', columns='adm_lvl', values='lvl_pcode_len')
segment_total_len = [int(i) for i in pivot_df.loc['AF'].tolist() if i>0]
segment_total_len

def split_pcode_struct(row):
    lvl=row['adm_lvl']
    pcode=row['pcode']
    c_code = row['pcode_prefix']
    segment_total_len = [int(i) for i in pivot_df.loc[c_code].tolist() if i>0]
    pcode_list = pcode.split()
    last_i=0
    newpcode=''
    for i in segment_total_len:
        x = pcode[last_i:i] 
        last_i = i
        if i <= len(pcode): #make sure the segment length doesn't exceed the length of the pcode
            newpcode = newpcode + x + '.'

    return newpcode[0:-1]

df_location['split_pcode'] = df_location.apply(split_pcode_struct, axis=1)
df_location

,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code,location_normalized,lvl_pcode_len,split_pcode
0,Afghanistan,AF,Khash,AF1705,2,en,khash,6,AF.17.05
1,Afghanistan,AF,Hazrat-e-Sultan,AF2002,2,en,hazratesultan,6,AF.20.02
2,Afghanistan,AF,Pul-e-Alam,AF0501,2,en,pulealam,6,AF.05.01
3,Afghanistan,AF,Mata Khan,AF1202,2,en,matakhan,6,AF.12.02
4,Afghanistan,AF,Gosfandi,AF2206,2,en,gosfandi,6,AF.22.06
...,...,...,...,...,...,...,...,...,...
2105,TURKEY,TU,YALOVA,TUR077,1,en,yalova,6,TUR.077
2106,TURKEY,TU,YOZGAT,TUR066,1,en,yozgat,6,TUR.066
2107,TURKEY,TU,ZONGULDAK,TUR067,1,en,zonguldak,6,TUR.067
2108,TURKEY,TU,TÜRKİYE,TUR,0,tr,trkiye,3,TUR


In [9]:
df_location.to_csv("c://temp//locations.csv", encoding='utf-8-sig', index=False)

# Now use the DF

In [11]:
def get_pcode_from_location(loc, country_prefix='XX'):

    if country_prefix != 'XX': #if the country prefix is set, limit search to that
        df_loc = df_location[df_location['pcode_prefix']]
    else:
        df_loc = df_location
        
    matches = df_loc['pcode'][df_loc['location_name'].str.lower() == loc.lower()].tolist()

    #if the match fails, try again on the normalized name
    if len(matches) == 0:
        #remove common variations in names that can cause misses
        n_loc = re.sub(r'[^a-zA-Z]', '', loc)

        #this will cause problems for non-English.. so if then len is 0, exit
        if len(n_loc) == 0:
            return []
            
        matches = df_loc['pcode'][df_loc['location_normalized'].str.lower() == n_loc.lower()].tolist()
        

    #now check results
    if len(matches) > 1:
        print(f"more than 1 matches... likely due to different granularity of entities with the same name (ie. Herat City in Herat Province) {matches}")
        return matches
    elif len(matches) == 1:
        return matches

    else:
        #couldn't find a match, do a fuzzy search
        compare_list = list(set(df_loc['location_name'].tolist()))
        possible_matches=[]
        for i in compare_list:
            if fuzz.ratio(loc,i) > 70:
                possible_matches.append(i)
                return [f"No exact match to '{loc}'. see if these alternative spellings are correct: {possible_matches}"]

    
    return []
                
        

#different scripts
script_check = ['Sharak-e-Hayratan','حضرت سلطان','Кальчикская','Mersin']
for r in script_check:
    res = get_pcode_from_location(r)
    print(f"searched for {r:<20} ->   {res}")

print()

#afghan references
afg_refs = ['Aliabad','Injil','Gulran','zindajan','kohsan'
            ,'Shakiban','Nazir Abad','Sanjab'
            ,'nazir','Ghar Moshak','Botan'
           ,'Hirat']
for r in afg_refs:
    res = get_pcode_from_location(r)
    print(f"searched for {r:20} : -> {res}")

searched for Sharak-e-Hayratan    ->   ['AF2116']
searched for حضرت سلطان           ->   ['AF2002']
searched for Кальчикская          ->   ['UA1414001']
searched for Mersin               ->   ['TUR033']

searched for Aliabad              : -> ['AF1903']
searched for Injil                : -> ['AF3202']
searched for Gulran               : -> ['AF3208']
searched for zindajan             : -> ['AF3205']
searched for kohsan               : -> ['AF3213']
searched for Shakiban             : -> ["No exact match to 'Shakiban'. see if these alternative spellings are correct: ['Shaki']"]
searched for Nazir Abad           : -> ["No exact match to 'Nazir Abad'. see if these alternative spellings are correct: ['Ghazi Abad']"]
searched for Sanjab               : -> ["No exact match to 'Sanjab'. see if these alternative spellings are correct: ['Panjab']"]
searched for nazir                : -> []
searched for Ghar Moshak          : -> []
searched for Botan                : -> []
more than 1 matches..

In [160]:
df_location

,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code,location_normalized,lvl_pcode_len,split_pcode
0,Afghanistan,AF,Khash,AF1705,2,en,khash,6,AF.17.05
1,Afghanistan,AF,Hazrat-e-Sultan,AF2002,2,en,hazratesultan,6,AF.20.02
2,Afghanistan,AF,Pul-e-Alam,AF0501,2,en,pulealam,6,AF.05.01
3,Afghanistan,AF,Mata Khan,AF1202,2,en,matakhan,6,AF.12.02
4,Afghanistan,AF,Gosfandi,AF2206,2,en,gosfandi,6,AF.22.06
...,...,...,...,...,...,...,...,...,...
7572,Ukraine,UA,Киев,UA80,1,ru,,4,UA.80
7573,Ukraine,UA,Севастополь,UA85,1,ru,,4,UA.85
7574,Ukraine,UA,Ukraine,UA,0,en,ukraine,2,UA
7575,Ukraine,UA,Україна,UA,0,ua,,2,UA


In [47]:
def get_adm_lvl_from_pcode(pcode):
    return list(set(df_location['adm_lvl'][df_location['pcode'] == pcode].tolist()))

print(get_adm_lvl_from_pcode('UA1414001'))

[3]


In [49]:
def get_name_in_lang(pcode, lang='en'):
    return list(set(df_location['location_name'][(df_location['pcode'] == pcode) & (df_location['lang_code'] == lang)].tolist()))

print(get_name_in_lang('UA1414001', 'en'))
print(get_name_in_lang('UA1414001', 'ru'))
print(get_name_in_lang('UA1414001', 'ua'))
print(get_name_in_lang('UA85', 'en'))

['Kalchytska']
['Кальчикская']
['Кальчицька']
['Sevastopol']


In [203]:
#get descendents of 
def get_descendents_of(pcode, lang='en', include_self=True):
    if include_self==True:
        return df_location[df_location['pcode'].str.contains(pcode) & (df_location['lang_code'] == lang)]
    else:
        return df_location[df_location['pcode'].str.contains(pcode) & (df_location['lang_code'] == lang)\
        & (df_location['pcode'] != pcode)]

        
get_descendents_of('UA1414001', include_self=False)


,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code,location_normalized,lvl_pcode_len,split_pcode


In [225]:
def get_admin_chain(pcode, lang='en'):
    split_pcode = df_location['split_pcode'][df_location['pcode'] == pcode].tolist()[0]
    levels = split_pcode.split(".")
    pc =''
    admin_chain = []
    #rebuild the pcode one level at a time
    for i in levels:
        pc = pc + i
        admin_chain.append(df_location['location_name'][(df_location['pcode'] == pc) & (df_location['lang_code'] == lang)].tolist()[0])

    return admin_chain
#UA1414001
get_admin_chain('UA0102013', 'en')

['Ukraine', 'Autonomous Republic of Crimea', 'Bakhchysaraiskyi', 'Holubynska']

In [222]:
search_loc = 'Kalchytska'
get_admin_chain('UA0102011')


['Ukraine', 'Autonomous Republic of Crimea', 'Bakhchysaraiskyi', 'Vilinska']

In [227]:
get_descendents_of('UA0116', include_self=False)

,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code,location_normalized,lvl_pcode_len,split_pcode
217,Ukraine,UA,Hvardiiska,UA0116001,3,en,hvardiiska,9,UA.01.16.001
218,Ukraine,UA,Hresivska,UA0116003,3,en,hresivska,9,UA.01.16.003
219,Ukraine,UA,Dobrivska,UA0116005,3,en,dobrivska,9,UA.01.16.005
220,Ukraine,UA,Donska,UA0116007,3,en,donska,9,UA.01.16.007
221,Ukraine,UA,Zhuravlivska,UA0116009,3,en,zhuravlivska,9,UA.01.16.009
222,Ukraine,UA,Kolchuhynska,UA0116011,3,en,kolchuhynska,9,UA.01.16.011
223,Ukraine,UA,Mazanska,UA0116013,3,en,mazanska,9,UA.01.16.013
224,Ukraine,UA,Mykolaivska,UA0116015,3,en,mykolaivska,9,UA.01.16.015
225,Ukraine,UA,Myrnivska,UA0116017,3,en,myrnivska,9,UA.01.16.017
226,Ukraine,UA,Molodizhnenska,UA0116019,3,en,molodizhnenska,9,UA.01.16.019


In [228]:
#test loading assertions

df_assertions = pd.read_excel("D:\\projects\\_external_files\\surveyor\\assertions.xlsx")
df_assertions


,pcode,attribute,numeric_value,date_of_observation,date_of_event,source_sentence
0,UA0102005,bicycle_bought,50,2023-11-18,2023-11-10,blah blah bla
1,UA0116021,bicycle_bought,500,2023-11-18,2023-11-10,blah blah bla
2,UA01,bicycle_bought,15000,2023-11-19,2023-11-10,a different source sentence


In [229]:
df_joined = df_location.merge(df_assertions, left_on='pcode', right_on='pcode')
df_joined[df_joined['lang_code'] == 'en']

,country,pcode_prefix,location_name,pcode,adm_lvl,lang_code,location_normalized,lvl_pcode_len,split_pcode,attribute,numeric_value,date_of_observation,date_of_event,source_sentence
0,Ukraine,UA,Bakhchysaraiska,UA0102005,3,en,bakhchysaraiska,9,UA.01.02.005,bicycle_bought,50,2023-11-18,2023-11-10,blah blah bla
3,Ukraine,UA,Novoandriivska,UA0116021,3,en,novoandriivska,9,UA.01.16.021,bicycle_bought,500,2023-11-18,2023-11-10,blah blah bla
6,Ukraine,UA,Autonomous Republic of Crimea,UA01,1,en,autonomousrepublicofcrimea,4,UA.01,bicycle_bought,15000,2023-11-19,2023-11-10,a different source sentence


## END

In [64]:
dfx = df.set_index(['adm0_pcode','adm1_pcode','adm2_pcode','adm3_pcode'])
dfx

adm3_en         adm3_ua  \
adm0_pcode adm1_pcode adm2_pcode adm3_pcode                                    
UA         UA01       UA0102     UA0102003      Aromatnenska   Ароматненська   
                                 UA0102005   Bakhchysaraiska  Бахчисарайська   
                                 UA0102009   Verkhorichenska  Верхоріченська   
                                 UA0102011          Vilinska       Вілінська   
                                 UA0102013        Holubynska     Голубинська   
...                                                      ...             ...   
           UA74       UA7410     UA7410035         Sednivska      Седнівська   
                                 UA7410037       Tupychivska     Тупичівська   
                                 UA7410039      Chernihivska    Чернігівська   
           UA80       UA8000     UA8000000              Kyiv            Київ   
           UA85       UA8500     UA8500000        Sevastopol     Севастополь   

                                                    adm3_ru  adm3_ref  \
adm0_pcode adm1_pcode adm2_pcode adm3_pcode                             
UA         UA01       UA0102     UA0102003    Ароматненский       NaN   
                                 UA0102005   Бахчисарайский       NaN   
                                 UA0102009   Верхореченский       NaN   
                                 UA0102011        Вилинский       NaN   
                                 UA0102013      Голубинский       NaN   
...                                                     ...       ...   
           UA74       UA7410     UA7410035       Седневская       NaN   
                                 UA7410037      Тупичевская       NaN   
                                 UA7410039     Черниговская       NaN   
           UA80       UA8000     UA8000000             Киев       NaN   
           UA85       UA8500     UA8500000      Севастополь       NaN   

                                             adm3alt1_en  adm3alt2_en  \
adm0_pcode adm1_pcode adm2_pcode adm3_pcode                             
UA         UA01       UA0102     UA0102003           NaN          NaN   
                                 UA0102005           NaN          NaN   
                                 UA0102009           NaN          NaN   
                                 UA0102011           NaN          NaN   
                                 UA0102013           NaN          NaN   
...                                                  ...          ...   
           UA74       UA7410     UA7410035           NaN          NaN   
                                 UA7410037           NaN          NaN   
                                 UA7410039           NaN          NaN   
           UA80       UA8000     UA8000000           NaN          NaN   
           UA85       UA8500     UA8500000           NaN          NaN   

                                             adm3alt1_ua  adm3alt2_ua  \
adm0_pcode adm1_pcode adm2_pcode adm3_pcode                             
UA         UA01       UA0102     UA0102003           NaN          NaN   
                                 UA0102005           NaN          NaN   
                                 UA0102009           NaN          NaN   
                                 UA0102011           NaN          NaN   
                                 UA0102013           NaN          NaN   
...                                                  ...          ...   
           UA74       UA7410     UA7410035           NaN          NaN   
                                 UA7410037           NaN          NaN   
                                 UA7410039           NaN          NaN   
           UA80       UA8000     UA8000000           NaN          NaN   
           UA85       UA8500     UA8500000           NaN          NaN   

                                             adm3alt1_ru  adm3alt2_ru  ...  \
adm0_pcode adm1_pcode adm2_pcode adm3_pcode                            ...   
UA    

In [95]:
dfx_nm = df.set_index(['adm0_en','adm1_en','adm2_en','adm3_en'])
dfx_nm = dfx_nm.sort_index()
dfx_nm['num_dead'] = 10
#dfx_nm.loc[('Ukraine','Autonomous Republic of Crimea','Bakhchysaraiskyi','Aromatnenska')]['num_dead'] = 20
#dfx_nm.loc[('Ukraine','Autonomous Republic of Crimea','Bakhchysaraiskyi')][['num_dead']] = 200


In [97]:
dfx_nm[['num_dead']] #.loc[('Ukraine','Autonomous Republic of Crimea','Bakhchysaraiskyi')]

num_dead
adm0_en adm1_en                       adm2_en          adm3_en                  
Ukraine Autonomous Republic of Crimea Bakhchysaraiskyi Aromatnenska           10
                                                       Bakhchysaraiska        10
                                                       Dolynnenska            10
                                                       Holubynska             10
                                                       Kashtanivska           10
...                                                                          ...
        Zhytomyrska                   Zviahelskyi      Pishchivska            10
                                                       Stryivska              10
                                                       Yarunska               10
                                                       Yemilchynska           10
                                                       Zviahelska             10

[1769 rows x 1 columns]

In [77]:
dfx_nm['Ukraine']['Autonomous Republic of Crimea']['Bakhchysaraiskyi']['Aromatnenska']

<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
C:\Users\ericf\AppData\Local\Temp\ipykernel_15064\2770309817.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  dfx_nm[['Ukraine']['Autonomous Republic of Crimea']['Bakhchysaraiskyi']['Aromatnenska']]


TypeError: list indices must be integers or slices, not str

In [63]:
for i in dfx.index:
    print(i)

('UA', 'UA01', 'UA0102')
('UA', 'UA01', 'UA0104')
('UA', 'UA01', 'UA0106')
('UA', 'UA01', 'UA0108')
('UA', 'UA01', 'UA0110')
('UA', 'UA01', 'UA0112')
('UA', 'UA01', 'UA0114')
('UA', 'UA01', 'UA0116')
('UA', 'UA01', 'UA0118')
('UA', 'UA01', 'UA0120')
('UA', 'UA05', 'UA0502')
('UA', 'UA05', 'UA0504')
('UA', 'UA05', 'UA0506')
('UA', 'UA05', 'UA0508')
('UA', 'UA05', 'UA0510')
('UA', 'UA05', 'UA0512')
('UA', 'UA07', 'UA0702')
('UA', 'UA07', 'UA0704')
('UA', 'UA07', 'UA0706')
('UA', 'UA07', 'UA0708')
('UA', 'UA12', 'UA1202')
('UA', 'UA12', 'UA1204')
('UA', 'UA12', 'UA1206')
('UA', 'UA12', 'UA1208')
('UA', 'UA12', 'UA1210')
('UA', 'UA12', 'UA1212')
('UA', 'UA12', 'UA1214')
('UA', 'UA14', 'UA1402')
('UA', 'UA14', 'UA1404')
('UA', 'UA14', 'UA1406')
('UA', 'UA14', 'UA1408')
('UA', 'UA14', 'UA1410')
('UA', 'UA14', 'UA1412')
('UA', 'UA14', 'UA1414')
('UA', 'UA14', 'UA1416')
('UA', 'UA18', 'UA1802')
('UA', 'UA18', 'UA1804')
('UA', 'UA18', 'UA1806')
('UA', 'UA18', 'UA1808')
('UA', 'UA21', 'UA2102')
